In [ ]:
from pathlib import Path

from linopy import Model
import pandas as pd

# Load data

In [ ]:
MAX_TIMESTEPS = 500
assert MAX_TIMESTEPS % 2 == 0

In [ ]:
ROOT_DIR = Path.cwd()
DATA_DIR = ROOT_DIR / "data"

battery_parameters = pd.read_csv(
    DATA_DIR / "battery_parameters.csv",
    index_col=0,
)["Values"].to_dict()
half_hourly_market_series = pd.read_csv(
    DATA_DIR / "half-hourly-market.csv",
    index_col=0,
    skiprows=1,
    nrows=MAX_TIMESTEPS,
    names=["Price (£/MWh)"],
).iloc[:, 0]
half_hourly_market_series.index = pd.to_datetime(half_hourly_market_series.index, format="%d/%m/%Y %H:%M")
hourly_market_series = pd.read_csv(
    DATA_DIR / "hourly-market.csv",
    index_col=0,
    skiprows=1,
    nrows=MAX_TIMESTEPS/2,
    names=["Price (£/MWh)"],
).iloc[:, 0]
hourly_market_series.index = pd.to_datetime(hourly_market_series.index, format="%d/%m/%Y %H:%M")

aligned_hourly_market_series = pd.concat([
    hourly_market_series,
    hourly_market_series.set_axis(hourly_market_series.index + pd.to_timedelta(30, unit='m')),
]).sort_index()

# Form model - simple case, participating in one market

In [ ]:
m = Model(
    force_dim_names=True  # add this in later for safety
)

Index:

In [ ]:
time = pd.Index(half_hourly_market_series.index, name="time")

Coefficients:
$$
\begin{array}{lll}
    \text{30 min market price at time $t$} & p_{30, t} & \\
    \text{60 min market price at time $t$} & p_{60, t} & \\
    \text{discharge efficiency} & e^d & = 0.95 \\
    \text{charge efficiency} & e^c & = 0.95 \\
    \text{minimum timestep duration} & \delta & = 30 \text{min} \\
\end{array}
$$

Variables:
$$
\begin{array}{lll}
    \text{discharging active} & s^d_t & \in \set{0, 1} \\
    \text{charging active} & s^c_t & \in \set{0, 1} \\
    \text{discharge rate to 30min market at time $t$} & r^d_{30,t} \\
    \text{charge rate to 30min market at time $t$} & r^c_{30,t} \\
    \text{discharge rate to 60min market at time $t$} & r^d_{60,t} \\
    \text{charge rate to 60min market at time $t$} & r^c_{60,t} \\
\end{array}
$$

where:
$$
\begin{array}{ll}
    0 \leq r^d_{30,t} \leq r^d_{max} & \text{(up to maximum discharge rate)} \\
    0 \leq r^c_{30,t} \leq r^c_{max} & \text{(up to maximum charge rate)} \\
    0 \leq r^d_{60,t} \leq r^d_{max} & \text{(up to maximum discharge rate)} \\
    0 \leq r^c_{60,t} \leq r^c_{max} & \text{(up to maximum charge rate)} \\
\end{array}
$$

In [ ]:
price_30min = half_hourly_market_series
price_60min = aligned_hourly_market_series

charge_efficiency = 1 - battery_parameters["Battery charging efficiency"]
discharge_efficiency = 1 - battery_parameters["Battery discharging efficiency"]

is_charging = m.add_variables(
    binary=True,
    coords=[time],
    name="is charging",
)
is_discharging = m.add_variables(
    binary=True,
    coords=[time],
    name="is discharging",
)
charge_rate_30 = m.add_variables(
    lower=0,
    upper=battery_parameters["Max charging rate"],
    coords=[time],
    name="charge rate 30",
)
discharge_rate_30 = m.add_variables(
    lower=0,
    upper=battery_parameters["Max discharging rate"],
    coords=[time],
    name="discharge rate 30",
)
charge_rate_60 = m.add_variables(
    lower=0,
    upper=battery_parameters["Max charging rate"],
    coords=[time],
    name="charge rate 60",
)
discharge_rate_60 = m.add_variables(
    lower=0,
    upper=battery_parameters["Max discharging rate"],
    coords=[time],
    name="discharge rate 60",
)

We wish to optimise profit:
$$
\max \sum_t^n \delta [ p_{30,t} (s^d_t r^d_{30,t} e^d - s^c_t r^c_{30,t} / e^c) + p_{60,t} (s^d_t r^d_{60,t} e^d - s^c_t r^c_{60,t} / e^c) ] 
$$

In [ ]:
m.add_objective(
    (
        price_30min * (discharge_rate_30 * discharge_efficiency - charge_rate_30 / charge_efficiency) + 
        price_60min * (discharge_rate_60 * discharge_efficiency - charge_rate_60 / charge_efficiency)
    ),
    sense="max",
)

subject to:
$$
\begin{array}{ll}
s^d_t + s^c_t \leq & \text{charging and discharging cannot occur simultaneously} \\
r^d_{30,t} + r^d_{60,t} \leq s^d_t r^d_{max} & \text{total max discharge rate} \\
r^c_{30,t} + r^c_{60,t} \leq s^c_t r^c_{max} & \text{total max charge rate} \\
\delta (r^d_{30,t} + r^d_{60,t}) \leq E_t & \text{cannot discharge more than the stored energy in a given timestep} \\
\delta (r^c_{30,t} + r^c_{60,t}) \leq E_{max} - E_t & \text{cannot charge more than the remaining capacity in a given timestep} \\
E_t = E_{init} + \delta \sum_{i=0}^{i=t-1} (r^c_i/e^c - r^d_i) & \text{stored energy is the cumulative sum of all prior charging and discharging events} \\
0 \leq E_t \leq E_t^{max} & \text{(up to maximum storage capacity)}
\end{array}
$$

where:
$$
\begin{array}{ll}
    \text{duration of timestep} & \delta \\
    \text{stored energy at time $t$} & E_t \\
    \text{maximum storage volume} & E_{max} \\
    \text{initial stored energy} & E_{init} \\
\end{array}
$$

In [ ]:
timestep_duration = 0.5
initial_stored_energy = 0.0
max_stored_energy = battery_parameters["Max storage volume"]
stored_energy = (
    initial_stored_energy + 
    timestep_duration * ((charge_rate_30 + charge_rate_60) * charge_efficiency - (discharge_rate_30 + discharge_rate_60)).shift(time=1).cumsum()
)

In [ ]:
m.add_constraints(is_charging + is_discharging <= 1, name="charge/discharge exclusive");

In [ ]:
m.add_constraints(charge_rate_30 + charge_rate_60 <= is_charging * battery_parameters["Max charging rate"], name="max charge rate");

In [ ]:
m.add_constraints(discharge_rate_30 + discharge_rate_60 <= is_discharging * battery_parameters["Max discharging rate"], name="max discharge rate");

In [ ]:
m.add_constraints(timestep_duration * (discharge_rate_30 + discharge_rate_60) <= stored_energy, name="available stored energy");

In [ ]:
m.add_constraints(timestep_duration * (charge_rate_30 + charge_rate_60) <= max_stored_energy - stored_energy, name="available storage capacity");

In [ ]:
m.add_constraints(charge_rate_60[::2] == charge_rate_60[1::2], name="60 min charge full hour");
m.add_constraints(discharge_rate_60[::2] == discharge_rate_60[1::2], name="60 min discharge full hour");

## Solve it

In [ ]:
m.solve(solver_name="highs")

In [ ]:
m.solution

# Validation

In [ ]:
assert ((charge_rate_30.solution * discharge_rate_30.solution).round(8) == 0).all(), "30min charge * discharge non zero"
assert ((charge_rate_60.solution * discharge_rate_60.solution).round(8) == 0).all(), "60min charge * discharge non zero"

In [ ]:
df = pd.DataFrame({
    "price 30": price_30min,
    "price 60": price_60min,
    "charge rate 30": charge_rate_30.solution.values,
    "discharge rate 30": discharge_rate_30.solution.values,
    "charge rate 60": charge_rate_60.solution.values,
    "discharge rate 60": discharge_rate_60.solution.values,
    "stored energy": stored_energy.solution.values,
    "energy added": (charge_rate_30.solution.values + charge_rate_60.solution.values) * charge_efficiency * timestep_duration,
    "energy removed":(discharge_rate_30.solution.values + discharge_rate_60.solution.values) * timestep_duration,
    "energy sold": (discharge_rate_30.solution.values + discharge_rate_60.solution.values) * timestep_duration * discharge_efficiency,
}, index=time)

df.head(10)

# Visualisation

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
fig, (axis_price, axis_energy, axis_flow_30, axis_flow_60) = plt.subplots(4, figsize=(12, 6), sharex=True)
axis_energy.set_ylim(0, 1.1*battery_parameters["Max storage volume"])
axis_flow_30.set_ylim(-1.25*battery_parameters["Max discharging rate"], 1.25*battery_parameters["Max charging rate"])
axis_price.set_ylabel("£")
axis_energy.set_ylabel("E")
axis_flow_30.set_ylabel("$r_{30}$")
axis_flow_60.set_ylabel("$r_{60}$")
axis_price.step(time, half_hourly_market_series, color="orange", label="30min", where="post")
axis_price.step(time, aligned_hourly_market_series, color="purple", label="60min", where="post")
axis_price.legend()
axis_energy.fill_between(
    time,
    stored_energy.solution.values,
    color="blue",
)
axis_flow_30.fill_between(
    time,
    m.solution.variables["charge rate 30"],
    step="post",
    color="green",
    label="charging",
)
axis_flow_30.fill_between(
    time,
    -m.solution.variables["discharge rate 30"],
    step="post",
    color="red",
    label="discharging",
)
axis_flow_30.legend()
axis_flow_60.fill_between(
    time,
    m.solution.variables["charge rate 60"],
    step="post",
    color="green",
    label="charging",
)
axis_flow_60.fill_between(
    time,
    -m.solution.variables["discharge rate 60"],
    step="post",
    color="red",
    label="discharging",
)
axis_flow_60.legend()